In [ ]:
is_colab = 'google.colab' in str(get_ipython())
if is_colab:
    !pip install neuron
    !git clone https://github.com/davidfague/Neural-Modeling.git -b cleanup
    %cd Neural-Modeling/scripts
    !pip install neuron_reduce
    !pip install ipympl

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../Modules/")

from Modules.simulation import Simulation
from Modules.cell_builder import SkeletonCell, CellBuilder
from Modules.constants import HayParameters

import os
from neuron import h

from logger import Logger # type: ignore

import numpy as np

In [ ]:
%cd ../scripts

In [ ]:
try:
    os.system("rm -r x86_64/")
except:
    pass

In [5]:
# plot FI with synapses and no mapping
import os
import numpy as np

# Define the template for common attributes
common_attributes = {
    'h_tstop': 2000,
    'all_synapses_off': False,
    'CI_on': True,
    'h_i_duration': 1950,
    'h_i_delay': 50,
    'synapse_mapping': False
}

# Define the varying attributes for each sim_name, without including 'sim_name' key
varying_attributes = [
    {'base_sim_name': 'Complex', 'reduce_cell_selective': False},
    {'base_sim_name': 'Basals', 'reduce_basals': True},
    {'base_sim_name': 'Tufts', 'reduce_tufts': True},
    {'base_sim_name': 'Obliques', 'reduce_obliques': True},
    {'base_sim_name': 'All', 'reduce_obliques': True, 'reduce_tufts': True, 'reduce_basals': True}
]

# Generate the list of HayParameters with updated sim_names
all_parameters = [
    HayParameters(
        **common_attributes,
        h_i_amplitude=round(amp, 1),
        sim_name=f"{attributes['base_sim_name']}_amp_{round(amp, 1)}",
        **{k: v for k, v in attributes.items() if k != 'base_sim_name'}
    )
    for amp in np.arange(-2, 2.1, 0.2)
    for attributes in varying_attributes
]

# Initialize simulation
sim = Simulation(SkeletonCell.Hay, title='FI_SynapsesNoMapping')

# Submit jobs to simulation
for parameters in all_parameters:
    sim.submit_job(parameters)

# Remove directory if it exists
try:
    os.system("rm -r x86_64/")
except:
    pass

# Run the simulation
sim.run()

# Create directory for saving results
output_dir = f"../notebooks/{sim.title}"
os.makedirs(output_dir, exist_ok=True)

# Get all parameter paths
all_parameters_paths = [parameters.path for parameters in all_parameters]

# Plot results and save to the directory
plot_command = f"python3 plot_fi.py -d {sim.path} -s {output_dir}"
os.system(plot_command)